In [232]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import re

In [233]:
df_dayseq1 = pd.read_csv(r"Annotations/daySequence1/frameAnnotationsBULB.csv", sep=';').sample(frac=1)
df_dayseq2 = pd.read_csv(r"Annotations/daySequence2/frameAnnotationsBULB.csv", sep=';').sample(frac=1)
df_nightseq1 = pd.read_csv(r"Annotations/nightSequence1/frameAnnotationsBULB.csv", sep=';').sample(frac=1)
df_nightseq2 = pd.read_csv(r"Annotations/nightSequence2/frameAnnotationsBULB.csv", sep=';').sample(frac=1)

In [234]:
# df_dayseq1.head(1000).sort_values(by=["Annotation tag"], ascending=False)
# df_dayseq1.groupby('Annotation tag').count()
num_of_warning = len(df_dayseq1[df_dayseq1['Annotation tag'] == 'warning']) + len(df_dayseq2[df_dayseq2['Annotation tag'] == 'warning']) + len(df_nightseq1[df_nightseq1['Annotation tag'] == 'warning']) + len(df_nightseq2[df_nightseq2['Annotation tag'] == 'warning']) - 1 # make it divisible by 4 (number of dataframes)


In [235]:
paths = [
    (df_dayseq1, "Filename", "dayTest", "daySequence1/frames"),
    (df_dayseq2, "Filename", "dayTest", "daySequence2/frames"),
    (df_nightseq1, "Filename", "nightTest", "nightSequence1/frames"),
    (df_nightseq2, "Filename", "nightTest", "nightSequence2/frames"),
]

In [236]:
def get_data(df, row_name, replace_param1, replace_param2, class_name, num_entries):
    training_data = []
    IMG_SIZE = 170
    count = 0
    
    if class_name == "stop":
        class_num = 0
    elif class_name == "warning":
        class_num = 1
    elif class_name == "go":
        class_num = 2
        
    dataframe = df[df['Annotation tag'] == class_name].sample(frac=1)
    
    for index, row in dataframe.iterrows():
        path = row[row_name].replace(replace_param1, replace_param2)
        
        img_array = cv2.imread(path)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE))
        training_data.append([new_array, class_num])
        count += 1
        
        if count == num_entries:
            break
            
    return training_data

In [237]:
data = []

for idx, item in enumerate(paths):
    data.append(get_data(paths[idx][0], paths[idx][1], paths[idx][2], paths[idx][3], 'stop', num_of_warning / 4))

for idx, item in enumerate(paths):
    data.append(get_data(paths[idx][0], paths[idx][1], paths[idx][2], paths[idx][3], 'warning', 100000)) # don't want this to break (want all warning entries)

for idx, item in enumerate(paths):
    data.append(get_data(paths[idx][0], paths[idx][1], paths[idx][2], paths[idx][3], 'go', num_of_warning / 4))

In [238]:
flattened_data = [item for items in data for item in items]

In [239]:
import random
random.shuffle(flattened_data)

In [240]:
stop_count = 0
go_count = 0
warning_count = 0

for features, label in flattened_data:
    if label == 0:
        stop_count += 1
    elif label == 1:
        warning_count += 1
    else:
        go_count +=1

In [244]:
print(len(flattened_data))
print(stop_count)
print(go_count)
print(warning_count)

4441
1480
1480
1481


In [245]:
X = []
y = []

for features, label in flattened_data:
    X.append(features)
    y.append(label)

In [246]:
print(len(X))
print(len(y))

4441
4441


In [247]:
import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()